In [26]:
import cv2
import csv
from tkinter import Tk, filedialog
import numpy as np  # Needed for rotation matrix computations

def select_rois_and_save_to_csv():

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()

    # Open a file dialog to select the video file
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi")])
    if not video_path:
        print("No video file selected.")
        return
    
    # Define the output CSV file
    output_csv = video_path.rsplit('.', 1)[0] + '_ROIs.csv'

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open the video file.")
        return

    # Read the first frame
    ret, first_frame = cap.read()
    cap.release()
    if not ret:
        print("Error: Unable to read the first frame.")
        return

    # Initialize variables
    clone = first_frame.copy()
    roi = []  # Current ROI [(x1, y1), (x2, y2)]
    dragging = [False]  # For moving ROI
    drag_start = None
    saved_rois = []
    resize_factor = 10  # Amount of change per scroll

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal roi, clone, drag_start, saved_rois

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            if not dragging[0]:
                dragging[0] = True
                roi = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(roi) == 1:
            clone = first_frame.copy()
            for r in saved_rois:
                cv2.rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2)
            cv2.rectangle(clone, roi[0], (x, y), (0, 255, 255), 2)

        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                roi.append((x, y))  # End point

        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(roi) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(roi) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            roi[0] = (roi[0][0] + dx, roi[0][1] + dy)
            roi[1] = (roi[1][0] + dx, roi[1][1] + dy)
            clone = first_frame.copy()
            for r in saved_rois:
                cv2.rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2)
            cv2.rectangle(clone, roi[0], roi[1], (0, 255, 255), 2)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(roi) == 2:
            dragging[0] = False

        # Resize ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(roi) == 2:
            x1, y1 = roi[0]
            x2, y2 = roi[1]
            if flags > 0:  # Scroll up
                x1 -= resize_factor
                y1 -= resize_factor
                x2 += resize_factor
                y2 += resize_factor
            else:  # Scroll down
                x1 += resize_factor
                y1 += resize_factor
                x2 -= resize_factor
                y2 -= resize_factor
            roi = [(x1, y1), (x2, y2)]
            clone = first_frame.copy()
            for r in saved_rois:
                cv2.rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2)
            cv2.rectangle(clone, roi[0], roi[1], (0, 255, 255), 2)

    # Set up the window and mouse callback
    cv2.namedWindow("Select ROIs")
    cv2.setMouseCallback("Select ROIs", handle_mouse)

    print("Instructions:")
    print("1. Left-click and drag to draw a rectangle.")
    print("2. Right-click and drag to move the rectangle.")
    print("3. Use the scroll wheel to resize the rectangle.")
    print("4. Press 'S' to save the current rectangle to the image.")
    print("5. Press 'Q' to quit and save all rectangles to the CSV file.")
    # print("6. Press 'Ctrl + Left-click and drag' to rotate the rectangle.")

    while True:
        cv2.imshow("Select ROIs", clone)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s') and len(roi) == 2:  # Save the ROI
            x1, y1 = roi[0]
            x2, y2 = roi[1]
            saved_roi = (min(x1, x2), min(y1, y2), abs(x2 - x1), abs(y2 - y1))
            saved_rois.append(saved_roi)
            last_saved_roi = saved_roi  # Store the last saved ROI for duplication
            print(f"Saved ROI: {saved_roi}")
            clone = first_frame.copy()
            for r in saved_rois:
                cv2.rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2)
            # We dont reset current ROI to be able to reuse it on another place of the frame.

        elif key == ord('q'):  # Quit and save
            print("Exiting and saving...")
            break

    cv2.destroyAllWindows()

    # Save the ROIs to a CSV file
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['x', 'y', 'width', 'height'])
        writer.writerows(saved_rois)

    print(f"ROIs saved to {output_csv}")




In [27]:
# Example usage:
select_rois_and_save_to_csv()

Instructions:
1. Left-click and drag to draw a rectangle.
2. Right-click and drag to move the rectangle.
3. Use the scroll wheel to resize the rectangle.
4. Press 'S' to save the current rectangle to the image.
5. Press 'Q' to quit and save all rectangles to the CSV file.
Saved ROI: (369, 121, 147, 246)
Saved ROI: (156, 224, 147, 246)
Saved ROI: (459, 403, 7, 106)
Saved ROI: (143, 172, 458, 90)
Saved ROI: (315, 92, 98, 270)
Exiting and saving...
ROIs saved to C:/Users/dhers/OneDrive - UBA/Club de la neurona/videos/example_NOR_ROIs.csv
